In [1]:
!pip install pyspark py4j

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean, when, count

In [3]:
# 1. Créer une session Spark
spark = SparkSession.builder \
    .appName("DataPreparation") \
    .getOrCreate()


In [4]:
from google.colab import files
uploaded = files.upload()

for filename in uploaded.keys():
    data_path = filename
    break

Saving cleaned_data.csv to cleaned_data.csv


In [5]:
df = spark.read.csv(data_path, header=True, inferSchema=True)

In [6]:
df.show(5)
df.printSchema()

+--------+--------+--------+---------+----------+---------+
|    Date|   Heure|Latitude|Longitude|Profondeur|Magnitude|
+--------+--------+--------+---------+----------+---------+
|9/8/2023|22:11:00|  30.998|   -8.444|       8.0|      7.0|
|9/8/2023|22:27:28|  30.986|   -8.412|       5.0|      3.9|
|9/8/2023|22:30:43|  30.995|   -8.421|       3.0|      5.7|
|9/8/2023|22:39:37|  30.859|   -8.427|       2.0|      3.4|
|9/8/2023|22:46:48|  30.894|   -8.379|       2.0|      3.7|
+--------+--------+--------+---------+----------+---------+
only showing top 5 rows

root
 |-- Date: string (nullable = true)
 |-- Heure: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Profondeur: double (nullable = true)
 |-- Magnitude: double (nullable = true)



In [7]:
# 4. Gérer les données manquantes
# Exemple : Remplacer les valeurs nulles par la moyenne pour les colonnes numériques
numeric_cols = [col_name for col_name, dtype in df.dtypes if dtype in ('int', 'double')]
for col_name in numeric_cols:
    mean_value = df.select(mean(col(col_name))).collect()[0][0]
    df = df.withColumn(col_name, when(col(col_name).isNull(), mean_value).otherwise(col(col_name)))

In [8]:
# 5. Supprimer les doublons
df = df.dropDuplicates()

In [9]:
# 6. Normaliser une colonne (exemple : mise à l'échelle d'une colonne numérique)
# Remplacez 'column_to_normalize' par le nom de la colonne à normaliser
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler

column_to_normalize = "Magnitude"
assembler = VectorAssembler(inputCols=[column_to_normalize], outputCol="features")
df = assembler.transform(df)

scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")
scaler_model = scaler.fit(df)
df = scaler_model.transform(df)

In [10]:
# Supprimer les colonnes intermédiaires si nécessaire
df = df.drop("features")

In [11]:
# 8. Afficher le résultat final
df.show(5)

+--------+--------+--------+---------+----------+---------+--------------------+
|    Date|   Heure|Latitude|Longitude|Profondeur|Magnitude|     scaled_features|
+--------+--------+--------+---------+----------+---------+--------------------+
|9/8/2023|23:25:18|  30.959|   -8.351|       2.0|      3.5|[0.4852941176470588]|
|9/9/2023| 9:52:55|  30.855|   -8.330|       6.0|      1.3|[0.16176470588235...|
|9/9/2023|12:28:06|  30.994|   -8.289|       3.0|      2.5| [0.338235294117647]|
|9/9/2023|21:46:58|  30.935|   -8.198|       4.0|      1.2|[0.14705882352941...|
|9/9/2023|22:52:32|  30.926|   -8.445|       3.0|      1.1|[0.13235294117647...|
+--------+--------+--------+---------+----------+---------+--------------------+
only showing top 5 rows



In [12]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

# Exemple de données de série temporelle (à remplacer par tes propres données)
dates = [
    datetime(2023, 9, 15),
    datetime(2023, 10, 1),
    datetime(2023, 10, 15),
    datetime(2023, 11, 1),
    datetime(2023, 11, 15),
    datetime(2023, 12, 1),
    datetime(2023, 12, 15),
    datetime(2023, 12, 31)
]
magnitudes = [3.5, 4.2, 3.8, 4.5, 4.1, 4.3, 4.0, 4.6]

# Créer une figure avec des sous-graphiques
fig = make_subplots(rows=3, cols=2, subplot_titles=['Série temporelle de la magnitude des tremblements de terre'])

# Ajouter le premier graphique
fig.add_trace(
    go.Scatter(x=dates, y=magnitudes, mode='lines+markers', name='Magnitude'),
    row=1, col=1
)

# Mettre à jour les titres et axes
fig.update_xaxes(title_text='Date', row=1, col=1, tickformat='%Y-%m-%d', dtick='M1')
fig.update_yaxes(title_text='Magnitude', row=1, col=1)

# Ajuster la disposition et le titre principal
fig.update_layout(height=800, width=1200, title_text='Visualisation de la Série Temporelle')

# Afficher le graphique
fig.show()

In [13]:
import plotly.graph_objects as go
import numpy as np

# Exemple de données de magnitudes (à remplacer par tes propres données)
magnitudes = [3.5, 4.2, 3.8, 4.5, 4.1, 4.3, 4.0, 4.6, 3.7, 3.9, 4.2, 4.4, 4.1, 4.3, 3.8, 4.0, 4.1, 4.5, 4.2, 4.3]

# Création de l'histogramme
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=magnitudes,
    nbinsx=20,
    histnorm='probability density',
    marker=dict(color='skyblue', line=dict(color='black', width=1))
))

# Mise à jour des titres et axes
fig.update_layout(
    title='Histogramme de la magnitude des séismes',
    xaxis_title='Magnitude',
    yaxis_title='Probabilité',
    bargap=0.2,
    height=600,
    width=800
)

# Afficher le graphique
fig.show()

In [14]:
import plotly.express as px

# Exemple de données de magnitudes (à remplacer par tes propres données)
magnitudes = [3.5, 4.2, 3.8, 4.5, 4.1, 4.3, 4.0, 4.6, 3.7, 3.9, 4.2, 4.4, 4.1, 4.3, 3.8, 4.0, 4.1, 4.5, 4.2, 4.3]

# Création de l'ECDF avec Plotly Express
fig = px.ecdf(
    x=magnitudes,
    title='Probabilité empirique de la magnitude des séismes',
    labels={'x':'Magnitude', 'y':'Probabilité'}
)

# Mise à jour des traces pour personnaliser la couleur et la largeur de la ligne
fig.update_traces(line=dict(color='green', width=2))

# Mise à jour de la disposition pour ajuster la taille
fig.update_layout(
    height=600,
    width=800,
    xaxis_title='Magnitude',
    yaxis_title='Probabilité',
)

# Afficher le graphique
fig.show()

In [15]:
import plotly.figure_factory as ff
import numpy as np

# Exemple de données de magnitudes (à remplacer par vos propres données)
magnitudes = [3.5, 4.2, 3.8, 4.5, 4.1, 4.3, 4.0, 4.6, 3.7, 3.9, 4.2, 4.4, 4.1, 4.3, 3.8, 4.0, 4.1, 4.5, 4.2, 4.3]

# Création de l'estimation de densité de noyau (KDE) avec Plotly
fig = ff.create_distplot([magnitudes], ['Magnitude'], colors=['orange'], show_hist=False, show_rug=False)

# Mise à jour des titres et axes
fig.update_layout(
    title='Estimation de densité de la magnitude des séismes',
    xaxis_title='Magnitude',
    yaxis_title='Densité',
    height=600,
    width=800
)

# Afficher le graphique
fig.show()


In [16]:
import plotly.express as px
import pandas as pd

# Exemple de données de magnitude et de profondeur (à remplacer par vos propres données)
data = {
    'Magnitude': [3.5, 4.2, 3.8, 4.5, 4.1, 4.3, 4.0, 4.6, 3.7, 3.9, 4.2, 4.4, 4.1, 4.3, 3.8, 4.0, 4.1, 4.5, 4.2, 4.3],
    'Profondeur': [10, 12, 15, 8, 9, 11, 14, 7, 13, 10, 12, 9, 8, 11, 13, 10, 12, 9, 14, 10]
}
df = pd.DataFrame(data)

# Création du boxplot avec Plotly Express
fig = px.box(df, y=['Magnitude', 'Profondeur'], points="all", color_discrete_sequence=px.colors.qualitative.Set3)

# Mise à jour des titres et axes
fig.update_layout(
    title='Boxplot de la magnitude et de la profondeur',
    yaxis_title='Valeurs',
    xaxis_title='',
    height=600,
    width=800
)

# Afficher le graphique
fig.show()


In [17]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Exemple de données de magnitude et de profondeur (à remplacer par vos propres données)
data = {
    'Date': [
        '2023-09-15', '2023-10-01', '2023-10-15', '2023-11-01', '2023-11-15',
        '2023-12-01', '2023-12-15', '2023-12-31', '2024-01-15', '2024-02-01',
        '2024-02-15', '2024-03-01', '2024-03-15', '2024-04-01', '2024-04-15',
        '2024-05-01', '2024-05-15', '2024-06-01', '2024-06-15', '2024-07-01'
    ],
    'Magnitude': [
        3.5, 4.2, 3.8, 4.5, 4.1, 4.3, 4.0, 4.6, 3.7, 3.9, 4.2, 4.4, 4.1, 4.3,
        3.8, 4.0, 4.1, 4.5, 4.2, 4.3
    ],
    'Profondeur': [
        10, 12, 15, 8, 9, 11, 14, 7, 13, 10, 12, 9, 8, 11, 13, 10, 12, 9, 14, 10
    ]
}
df = pd.DataFrame(data)

# Préparer les données pour la heatmap
heatmap_data = pd.pivot_table(df, values='Magnitude', index='Profondeur', columns='Date', aggfunc=np.mean)

# Création de la heatmap avec Plotly
fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,
    x=heatmap_data.columns,
    y=heatmap_data.index,
    colorscale='Viridis'
))

# Mise à jour des titres et axes
fig.update_layout(
    title='Heatmap de la profondeur et de la magnitude',
    xaxis_title='Date',
    yaxis_title='Profondeur',
    height=600,
    width=800
)

# Afficher le graphique
fig.show()


<ipython-input-17-a87f598d42c0>:24: FutureWarning:

The provided callable <function mean at 0x7dd6f4bdd5a0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.



In [18]:
import plotly.express as px
import pandas as pd

# Exemple de données de profondeur (à remplacer par vos propres données)
data = {
    'Profondeur': [
        10, 12, 15, 8, 9, 11, 14, 7, 13, 10, 12, 9, 8, 11, 13, 10, 12, 9, 14, 10
    ]
}
df = pd.DataFrame(data)

# Création de l'histogramme avec Plotly
fig = px.histogram(df, x='Profondeur', nbins=20, color_discrete_sequence=['green'])

# Mise à jour des titres et axes
fig.update_layout(
    title='Histogramme de la profondeur des tremblements de terre',
    xaxis_title='Profondeur',
    yaxis_title='Densité',
    height=600,
    width=800
)

# Afficher le graphique
fig.show()

In [19]:
import plotly.graph_objects as go
import pandas as pd

# Exemple de données (à remplacer par vos propres données)
data = pd.DataFrame({
    'Magnitude': [3.5, 4.2, 3.8, 4.5, 4.1],
    'Profondeur': [10, 15, 12, 18, 14],
    'Latitude': [30, 32, 31, 29, 33],
    'Longitude': [-120, -118, -119, -121, -117]
})

# Calculer la matrice de corrélation
correlation_matrix = data[['Magnitude', 'Profondeur', 'Latitude', 'Longitude']].corr()

# Créer la heatmap avec Plotly
fig = go.Figure(data=go.Heatmap(
        z=correlation_matrix.values,
        x=correlation_matrix.columns,
        y=correlation_matrix.columns,
        colorscale='Viridis',  # Utilisation de 'Viridis' au lieu de 'coolwarm'
        colorbar=dict(title='Corrélation')
))

# Personnalisation de l'aspect visuel et des annotations
fig.update_layout(
    title='Matrice de corrélation',
    xaxis_title='Variables',
    yaxis_title='Variables',
    height=600,
    width=800,
    font=dict(size=12),
)

# Afficher le graphique
fig.show()

In [20]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import warnings

# Ignorer les avertissements spécifiques
warnings.filterwarnings("ignore", message="Too few observations to estimate starting parameters for ARMA and trend.")
warnings.filterwarnings("ignore", message="Maximum Likelihood optimization failed to converge.")

# Exemple de données série temporelle (à remplacer par tes propres données)
serie = data['Magnitude']
serie.index = pd.date_range(start='2023-09-01', periods=len(serie), freq='D')  # Assurez-vous que l'index est une série temporelle

# Ajuster le modèle ARIMA
model = ARIMA(serie, order=(5, 1, 0))  # Exemple avec ARIMA(5,1,0)
fit_model = model.fit()

# Faire des prédictions pour les 10 prochaines périodes
predictions = fit_model.forecast(steps=10)

# Afficher les prédictions
print("Prédictions ARIMA:")
print(predictions)

Prédictions ARIMA:
2023-09-06    4.794847
2023-09-07    4.385631
2023-09-08    5.066928
2023-09-09    4.640810
2023-09-10    5.302612
2023-09-11    4.855370
2023-09-12    5.495219
2023-09-13    5.025948
2023-09-14    5.644274
2023-09-15    5.154474
Freq: D, Name: predicted_mean, dtype: float64


In [21]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Exemple de série temporelle
np.random.seed(0)
date_range = pd.date_range(start='2023-09-01', periods=100, freq='D')
serie = pd.Series(np.random.randn(len(date_range)), index=date_range)

# Exemple de prédictions (mêmes dates pour cet exemple)
predictions = pd.Series(np.random.randn(len(date_range)), index=date_range)

# Créer une figure Plotly
fig = go.Figure()

# Ajouter les données réelles à la figure
fig.add_trace(go.Scatter(x=serie.index, y=serie.values, mode='lines', name='Données réelles'))

# Ajouter les prédictions à la figure
fig.add_trace(go.Scatter(x=predictions.index, y=predictions.values, mode='lines', name='Prédictions', line=dict(color='red')))

# Personnaliser la mise en page et les titres
fig.update_layout(
    title='Prédictions ARIMA',
    xaxis_title='Date',
    yaxis_title='Magnitude',
    height=600,
    width=1000,
    font=dict(size=12),
    legend=dict(x=0, y=1.0, bgcolor='rgba(255, 255, 255, 0.5)', bordercolor='rgba(255, 255, 255, 0.5)')
)

# Afficher la figure
fig.show()

In [22]:
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
from sklearn.metrics import mean_squared_error

# Exemple de données (remplace-le par tes propres données)
train_data = pd.DataFrame({
    'Magnitude': [3.5, 4.2, 3.8, 4.5, 4.1],
    # Ajoute d'autres colonnes si nécessaire pour les variables exogènes
})

# Ajuster le modèle ARIMA avec un ensemble de données d'entraînement
model = ARIMA(train_data['Magnitude'], order=(1, 0, 0))  # Exemple avec ARIMA(1,0,0) sans différenciation
fit_model = model.fit()

# Prédire les valeurs pour les données de test (utilisation fictive pour l'exemple)
test_data = pd.DataFrame({
    'Magnitude': [4.3, 4.1, 4.6, 4.0, 4.4],
    # Assurez-vous que les données de test ont la même structure que les données d'entraînement
})

forecast = fit_model.forecast(steps=len(test_data))

# Calculer la RMSE
rmse = np.sqrt(mean_squared_error(test_data['Magnitude'], forecast))
print("RMSE du modèle ARIMA:", rmse)

RMSE du modèle ARIMA: 0.3166100156225418


In [23]:
import numpy as np
import plotly.graph_objects as go
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm

# Exemple de données (à remplacer par vos propres données)
np.random.seed(42)
train_data = np.random.randn(100)

# Ajuster le modèle ARIMA (exemple avec ARIMA(1,0,0))
model = ARIMA(train_data, order=(1, 0, 0))
fit_model = model.fit()

# Calculer les résidus
residuals = train_data - fit_model.fittedvalues

# Calculer l'ACF des résidus avec statsmodels
acf_values, confint = sm.tsa.stattools.acf(residuals, nlags=20, alpha=0.05)

# Créer une figure Plotly pour l'ACF des résidus
acf_fig = go.Figure()

# Ajouter les barres d'ACF avec Plotly
acf_fig.add_trace(go.Bar(x=np.arange(len(acf_values)), y=acf_values, marker_color='orange', name='ACF des résidus'))

# Ligne horizontale pour le seuil de significativité
acf_fig.add_shape(type="line",
                  x0=0,
                  y0=confint[1],
                  x1=len(acf_values) - 1,
                  y1=confint[1],
                  line=dict(color="red", width=2, dash="dash"),
                  name="Seuil supérieur de signification (95%)")

acf_fig.add_shape(type="line",
                  x0=0,
                  y0=confint[0],
                  x1=len(acf_values) - 1,
                  y1=confint[0],
                  line=dict(color="green", width=2, dash="dash"),
                  name="Seuil inférieur de signification (95%)")

# Mise à jour du layout de la figure
acf_fig.update_layout(title='ACF des résidus',
                      xaxis_title='Lags',
                      yaxis_title='Autocorrélation',
                      height=600,
                      width=800)

# Afficher la figure Plotly de l'ACF des résidus
acf_fig.show()

In [24]:
import numpy as np
import plotly.graph_objects as go

# Exemple de données (à remplacer par vos propres données)
np.random.seed(42)
train_data = np.random.randn(100)

# Ajuster le modèle ARIMA (exemple avec ARIMA(1,0,0))
model = ARIMA(train_data, order=(1, 0, 0))
fit_model = model.fit()

# Calculer les résidus
residuals = train_data - fit_model.fittedvalues

# Créer une figure Plotly pour la série temporelle des résidus
residuals_fig = go.Figure()

# Ajouter une trace de ligne pour la série temporelle des résidus
residuals_fig.add_trace(go.Scatter(x=np.arange(len(train_data)), y=residuals,
                                   mode='markers+lines', marker=dict(color='blue'),
                                   name='Série temporelle des résidus'))

# Personnalisation du layout de la figure
residuals_fig.update_layout(title='Série temporelle des résidus',
                            xaxis_title='Index',
                            yaxis_title='Résidus',
                            height=600,
                            width=800)

# Afficher la figure Plotly de la série temporelle des résidus
residuals_fig.show()


In [25]:
import numpy as np
import plotly.graph_objects as go

# Exemple de données (à remplacer par vos propres données)
np.random.seed(42)
train_data = np.random.randn(100)

# Ajuster le modèle ARIMA (exemple avec ARIMA(1,0,0))
model = ARIMA(train_data, order=(1, 0, 0))
fit_model = model.fit()

# Calculer les résidus
residuals = train_data - fit_model.fittedvalues

# Créer une figure Plotly pour l'histogramme des résidus
histogram_fig = go.Figure()

# Ajouter une trace d'histogramme pour les résidus
histogram_fig.add_trace(go.Histogram(x=residuals, nbinsx=20,
                                     marker_color='green',
                                     name='Histogramme des résidus'))

# Personnalisation du layout de la figure
histogram_fig.update_layout(title='Histogramme des résidus',
                            xaxis_title='Résidus',
                            yaxis_title='Fréquence',
                            height=600,
                            width=800)

# Afficher la figure Plotly de l'histogramme des résidus
histogram_fig.show()

In [26]:
import numpy as np
import plotly.graph_objects as go
from scipy import stats

# Exemple de données (à remplacer par vos propres données)
np.random.seed(42)
train_data = np.random.randn(100)

# Ajuster le modèle ARIMA (exemple avec ARIMA(1,0,0))
model = ARIMA(train_data, order=(1, 0, 0))
fit_model = model.fit()

# Calculer les résidus
residuals = train_data - fit_model.fittedvalues

# Calculer les quantiles des résidus
residuals_sorted = np.sort(residuals)
quantiles = np.arange(0.5, len(residuals_sorted), step=1) / len(residuals_sorted) * 100
theoretical_quantiles = stats.norm.ppf(quantiles / 100)

# Créer une figure Plotly pour le QQ plot des résidus
qqplot_fig = go.Figure()

# Ajouter une trace de scatterplot pour le QQ plot
qqplot_fig.add_trace(go.Scatter(x=theoretical_quantiles,
                                y=np.percentile(residuals_sorted, quantiles),
                                mode='markers',
                                marker=dict(color='blue'),
                                name='QQ Plot des résidus'))

# Ligne diagonale pour représenter une distribution normale
x_line = np.linspace(np.min(theoretical_quantiles), np.max(theoretical_quantiles), 100)
y_line = np.linspace(np.min(theoretical_quantiles), np.max(theoretical_quantiles), 100)
qqplot_fig.add_trace(go.Scatter(x=x_line, y=y_line,
                                mode='lines',
                                line=dict(color='red', dash='dash'),
                                showlegend=False))

# Personnalisation du layout de la figure
qqplot_fig.update_layout(title='QQ Plot des résidus',
                         xaxis_title='Quantiles théoriques (distribution normale)',
                         yaxis_title='Quantiles des résidus',
                         height=600,
                         width=800)

# Afficher la figure Plotly du QQ plot des résidus
qqplot_fig.show()

In [27]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import plotly.graph_objects as go

# Création du DataFrame avec les dates mises à jour
dates = pd.date_range(start='2023-09-01', end='2023-12-31', freq='D')
data = {
    'Datetime': dates,
    'Magnitude': np.random.uniform(3.0, 7.0, len(dates))  # Génération de données aléatoires pour Magnitude
}

df = pd.DataFrame(data)
df.set_index('Datetime', inplace=True)
df.index = pd.DatetimeIndex(df.index, freq='D')  # Spécification explicite de la fréquence

# On va se concentrer sur la variable Magnitude
ts = df['Magnitude']

# Ajustement du modèle SARIMA
model = sm.tsa.statespace.SARIMAX(ts, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7))
model_fit = model.fit(disp=False)

# Génération des prévisions dynamiques jusqu'à la fin de l'année 2023 et au-delà
forecast_period = 30  # 30 jours de prévisions supplémentaires
dynamic_forecast = model_fit.get_forecast(steps=forecast_period)
mean_forecast = dynamic_forecast.predicted_mean
conf_int = dynamic_forecast.conf_int()

# Créer une nouvelle série temporelle avec les prévisions ajoutées
forecast_index = pd.date_range(start=ts.index[-1] + pd.Timedelta(days=1), periods=forecast_period, freq='D')
mean_forecast.index = forecast_index
conf_int.index = forecast_index

# Conversion des données pour Plotly
ts_plotly = go.Scatter(x=ts.index, y=ts, mode='lines', name='Données réelles', opacity=0.6)
forecast_plotly = go.Scatter(x=mean_forecast.index, y=mean_forecast, mode='lines', name='Prévisions dynamiques', line=dict(color='red'))
conf_int_plotly = go.Scatter(
    x=forecast_index.tolist() + forecast_index.tolist()[::-1],
    y=conf_int.iloc[:, 0].tolist() + conf_int.iloc[:, 1].tolist()[::-1],
    fill='toself',
    fillcolor='rgba(255, 0, 0, 0.2)',
    line=dict(color='rgba(255, 0, 0, 0)'),
    hoverinfo="skip",
    showlegend=False
)

# Création de la figure Plotly
fig = go.Figure([ts_plotly, forecast_plotly, conf_int_plotly])

# Personnalisation du layout
fig.update_layout(title='Prévisions dynamiques de Magnitude - Graphique en lignes',
                  xaxis_title='Date',
                  yaxis_title='Magnitude',
                  width=900,
                  height=500)

# Affichage du graphique Plotly
fig.show()